In [1]:
# url <- 'https://www.draftkings.com/lineup/getavailableplayerscsv?contestTypeId=84&draftGroupId=36302'
# df <- read.csv(url,stringsAsFactors=FALSE)
# head(df)

In [2]:
df <- read.csv('Data/Outlaw_projections_20200625.csv',stringsAsFactors=FALSE)

#only keep odds less than 1000
df <- subset(df,Odds<=1000)
tail(df)

,Name...ID,Name,Salary,Odds,Points,Win,Tee,Hole,tee_delay,Points_adj
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,<dbl>
44,Alex Lee (14679243) (LOCKED),Alex Lee,6400,750,-8.807209,0.001331558,9:00 AM,1,1.5000000,-11.577209
45,Ryan Schmitz (14679244) (LOCKED),Ryan Schmitz,6300,300,-4.135991,0.003322259,1:10 PM,1,5.6666667,-4.135991
46,Nick Cantlay (14679245) (LOCKED),Nick Cantlay,6200,400,-5.997651,0.002493766,8:10 AM,1,0.6666667,-8.767651
47,Chase Carlson (14679246) (LOCKED),Chase Carlson,6200,500,-7.222029,0.001996008,12:40 PM,1,5.1666667,-7.222029
48,Steve Sear (14679247) (LOCKED),Steve Sear,6100,750,-8.807209,0.001331558,1:50 PM,10,6.3333333,-7.027209
49,Li Wang (14679248) (LOCKED),Li Wang,6000,600,-8.021154,0.001663894,1:30 PM,10,6.0000000,-6.241154


In [3]:
#Add column of ones
df['ones'] <- 1
df$ones <- as.integer(df$ones)
head(df)

Name...ID,Name,Salary,Odds,Points,Win,Tee,Hole,tee_delay,Points_adj,ones
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<int>
John Catlin (14679194) (LOCKED),John Catlin,10700,8,7.383962,0.11111111,8:40 AM,10,1.166667,6.393962,1
Matt Picanso (14679195) (LOCKED),Matt Picanso,10400,14,7.023074,0.06666667,2:00 PM,10,6.500000,8.803074,1
Mark Anguiano (14679196) (LOCKED),Mark Anguiano,10200,16,6.904185,0.05882353,1:50 PM,1,6.333333,6.904185,1
Alistair Docherty (14679197) (LOCKED),Alistair Docherty,9900,20,6.668501,0.04761905,1:10 PM,10,5.666667,8.448501,1
Norman Xiong (14679198) (LOCKED),Norman Xiong,9700,28,6.205432,0.03448276,1:10 PM,10,5.666667,7.985432,1
Brady Calkins (14679200) (LOCKED),Brady Calkins,9300,16,6.904185,0.05882353,9:30 AM,1,2.000000,4.134185,1


In [4]:
library('lpSolve')
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [7]:
#Practice finding optimal team
# Set coefficients of the objective function
f.obj <- df$Points_adj

# Set matrix corresponding to coefficients of constraints by rows
#f*dk_salary <= 50000
#f*ones == 6
f.con <- t(data.matrix(df[c('Salary','ones')]))

# Set unequality/equality signs
f.dir <- c("<=",
           "==")

# Set right hand side coefficients
f.rhs <- c(50000,
           6)

# # Variables final values
opt_team <- lp("max", f.obj, f.con, f.dir, f.rhs, all.bin = TRUE)$solution
df['opt_team'] <- opt_team
filter(df,opt_team==1)

Name...ID,Name,Salary,Odds,Points,Win,Tee,Hole,tee_delay,Points_adj,ones,opt_team
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>
Alistair Docherty (14679197) (LOCKED),Alistair Docherty,9900,20,6.668501,0.04761905,1:10 PM,10,5.666667,8.448501,1,1
Norman Xiong (14679198) (LOCKED),Norman Xiong,9700,28,6.205432,0.03448276,1:10 PM,10,5.666667,7.985432,1,1
Andrew Funk (14679211) (LOCKED),Andrew Funk,8200,40,5.531240,0.02439024,1:50 PM,1,6.333333,5.531240,1,1
Zach Smith (14679212) (LOCKED),Zach Smith,8100,20,6.668501,0.04761905,1:00 PM,1,5.500000,6.668501,1,1
Ryan Gronlund (14679220) (LOCKED),Ryan Gronlund,7500,40,5.531240,0.02439024,8:30 AM,10,1.000000,4.541240,1,1
George Markham (14679238) (LOCKED),George Markham,6600,100,2.508003,0.00990099,8:50 AM,10,1.333333,1.518003,1,1


In [18]:
#Find top N teams
num_teams = 6
df_now <- df
to_remove = c()
for (n in to_remove) {
    df_now <- filter(df_now,Name!=n) 
}
#Max sal
max_sal <- 99900
min_sal <- 6000
df_now <- subset(df_now,Salary<=max_sal)
df_now <- subset(df_now,Salary>=min_sal)
base_exposure <- 50
exposure_gap <- 5

for (i in seq(1:num_teams)) {
    # Set coefficients of the objective function
    f.obj <- df_now$Points
    #f.obj <- df_now$Points_adj

    # Set matrix corresponding to coefficients of constraints by rows
    #f*dk_salary <= 50000
    #f*ones == 6
    f.con <- t(data.matrix(df_now[c('Salary','ones')]))
    
    # Set unequality/equality signs
    f.dir <- c("<=",
               "==")
    
    # Set right hand side coefficients
    f.rhs <- c(50000,
               6)

    # # Variables final values
    #opt_val <- lp("max", f.obj, f.con, f.dir, f.rhs, int.vec = 1:4, all.bin = TRUE)$objective
    opt_team <- lp("max", f.obj, f.con, f.dir, f.rhs, int.vec = 1:4, all.bin = TRUE)$solution
    df_now['opt_team'] <- opt_team
    df_opt <- filter(df_now,opt_team==1)
    
    #Print current best team
    print(paste('Team ',i,':',sep=""))
    print(paste(df_opt$Name))
    
    #Add best team to df_teams
    if (i==1) {
        df_teams <- df_opt[c('Name...ID','Salary','Tee','Hole')]
        df_teams['Exposure'] <- base_exposure
    } else {
        df_teams_new <- df_opt[c('Name...ID','Salary','Tee','Hole')]
        df_teams_new['Exposure'] <- base_exposure - (i-1)*exposure_gap
        df_teams <- rbind(df_teams,df_teams_new)
    }
    
    #Remove current opt team
    df_now <- filter(df_now,opt_team==0)
}

[1] "Team 1:"
[1] "Brady Calkins"  "David Gazzolo"  "Andrew Funk"    "Zach Smith"    
[5] "Zander Winston" "Ryan Gronlund" 
[1] "Team 2:"
[1] "John Catlin"     "Eric Sugimoto"   "Matt Marshall"   "Michael McGowan"
[5] "Riley Elmes"     "Ben Geyer"      
[1] "Team 3:"
[1] "Mark Anguiano"     "Alistair Docherty" "Andrew Yun"       
[4] "Bryan Martin"      "Jino Sohn"         "George Markham"   
[1] "Team 4:"
[1] "Matt Picanso"    "Derek Bayley"    "Noah Hofman"     "Mario Beltran"  
[5] "Matt Snyder"     "Austin Bautista"
[1] "Team 5:"
[1] "Norman Xiong"      "Blake Abercrombie" "Jared du Toit"    
[4] "Chad Hambright"    "Cole Madey"        "Dalan Refioglu"   
[1] "Team 6:"
[1] "Michael Weaver" "Derek Castillo" "Sam Gillis"     "John Greco"    
[5] "Grant Booth"    "Jared Sawada"  


In [19]:
teams_to_keep = 2
low_exposure = base_exposure - (teams_to_keep-1)*exposure_gap
df_teams <- filter(df_teams,Exposure >= low_exposure)
df_teams <- df_teams[order(-df_teams$Salary),]
df_teams

,Name...ID,Salary,Tee,Hole,Exposure
,<chr>,<int>,<chr>,<int>,<dbl>
7,John Catlin (14679194),10700,8:40 AM,10,45
1,Brady Calkins (14679200),9300,9:30 AM,1,50
2,David Gazzolo (14679206),8700,1:00 PM,1,50
8,Eric Sugimoto (14679207),8600,8:40 AM,1,45
9,Matt Marshall (14679209),8400,1:20 PM,1,45
3,Andrew Funk (14679211),8200,1:50 PM,1,50
4,Zach Smith (14679212),8100,1:00 PM,1,50
5,Zander Winston (14679213),8000,9:20 AM,10,50
10,Michael McGowan (14679218),7600,8:50 AM,10,45


In [20]:
#Add more players
more_names = c('Sam Gillis',"Mario Beltran", "Mark Anguiano","Alistair Docherty")
df_more <- subset(df,Name %in% more_names)[c('Name...ID','Salary','Tee','Hole')]
df_more['Exposure'] = low_exposure-1
df_more

,Name...ID,Salary,Tee,Hole,Exposure
,<chr>,<int>,<chr>,<int>,<dbl>
3,Mark Anguiano (14679196),10200,1:50 PM,1,44
4,Alistair Docherty (14679197),9900,1:10 PM,10,44
19,Mario Beltran (14679214),7900,9:00 AM,1,44
29,Sam Gillis (14679224),7300,2:10 PM,10,44


In [21]:
#Add more players and rsete exposure
df_teams <- rbind(df_teams,df_more)
df_teams <- df_teams[order(-df_teams$Salary),]

#replace exposure with probabilities
df_teams$Exposure <- df_teams$Exposure / sum(df_teams$Exposure)
df_teams

,Name...ID,Salary,Tee,Hole,Exposure
,<chr>,<int>,<chr>,<int>,<dbl>
7,John Catlin (14679194),10700,8:40 AM,10,0.06032172
31,Mark Anguiano (14679196),10200,1:50 PM,1,0.05898123
41,Alistair Docherty (14679197),9900,1:10 PM,10,0.05898123
1,Brady Calkins (14679200),9300,9:30 AM,1,0.06702413
2,David Gazzolo (14679206),8700,1:00 PM,1,0.06702413
8,Eric Sugimoto (14679207),8600,8:40 AM,1,0.06032172
9,Matt Marshall (14679209),8400,1:20 PM,1,0.06032172
3,Andrew Funk (14679211),8200,1:50 PM,1,0.06702413
4,Zach Smith (14679212),8100,1:00 PM,1,0.06702413


In [22]:
#PRactice making df_lineups
col_names = c('G1','G2','G3','G4','G5','G6')
df_lineups = data.frame('test','test','test','test','test','test',stringsAsFactors=FALSE)
names(df_lineups) <- col_names
df_lineups

G1,G2,G3,G4,G5,G6
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
test,test,test,test,test,test


In [23]:
#Practice grabbing 6 players
curr_team <- sample(df_teams$Name...ID,size=6,replace=FALSE,prob = df_teams$Exposure)
df_now <- subset(df_teams,Name...ID %in% curr_team)
df_now <- df_now[order(-df_now$Salary),]
curr_team <- df_now$Name...ID
curr_team

[1] "John Catlin (14679194)"     "Brady Calkins (14679200)"  
[3] "David Gazzolo (14679206)"   "Eric Sugimoto (14679207)"  
[5] "Michael McGowan (14679218)" "Ryan Gronlund (14679220)"

In [24]:
#Add to df_teams$Actual
df_teams$Name...ID %in% curr_team

[1]  TRUE FALSE FALSE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE  TRUE
[13] FALSE  TRUE FALSE FALSE

In [25]:
num_teams <- 35
n <- 0
min_sal <- 49400
max_sal <- 50000

#Reset df_teams
df_teams['Actual'] <- 0

#Reset df_lineups
col_names = c('G1','G2','G3','G4','G5','G6')
df_lineups = data.frame('test','test','test','test','test','test',stringsAsFactors=FALSE)
names(df_lineups) <- col_names

while(n<num_teams) {
    #Pick random team
    curr_team <- sample(df_teams$Name...ID,size=6,replace=FALSE,prob = df_teams$Exposure)
    df_now <- subset(df_teams,Name...ID %in% curr_team)
    #Get team in order of descending salary
    df_now <- df_now[order(-df_now$Salary),]
    curr_team <- df_now$Name...ID
    if (sum(df_now$Salary)>=min_sal & sum(df_now$Salary)<=max_sal) {
        #print(curr_team)
        df_lineups <- rbind(df_lineups,curr_team)
        n <- n+1
        if (n%%10 == 0) {
            print(n)
        }
        #Update df_teams
    }
}
subset(df_lineups,G1=="Matt Picanso (14671270)")

[1] 10
[1] 20
[1] 30


G1,G2,G3,G4,G5,G6
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


In [26]:
head(df_lineups)

G1,G2,G3,G4,G5,G6
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
test,test,test,test,test,test
John Catlin (14679194),David Gazzolo (14679206),Matt Marshall (14679209),Ryan Gronlund (14679220),Sam Gillis (14679224),Ben Geyer (14679229)
Brady Calkins (14679200),David Gazzolo (14679206),Eric Sugimoto (14679207),Zach Smith (14679212),Michael McGowan (14679218),Riley Elmes (14679219)
Alistair Docherty (14679197),Eric Sugimoto (14679207),Matt Marshall (14679209),Mario Beltran (14679214),Michael McGowan (14679218),Sam Gillis (14679224)
John Catlin (14679194),Andrew Funk (14679211),Zach Smith (14679212),Zander Winston (14679213),Sam Gillis (14679224),Ben Geyer (14679229)
Mark Anguiano (14679196),Eric Sugimoto (14679207),Zach Smith (14679212),Zander Winston (14679213),Michael McGowan (14679218),Sam Gillis (14679224)


In [27]:
write.csv(df_lineups,'Lineups/DKEntries_Outlaw.csv')